# The Pepper Viz, Part 1: Data Preparation

### The goal of this project is to create visualizations around spicy peppers. That all starts with a set of data for use to work with and visualize. In this notebook we will:

* Scrape our data source website
* Format the data
* Export it into a static, easy to use format

In [1]:
import numpy as np
import pandas as pd
import requests

## Scraping the Data

#### First we need to scrape a website with the available information. For this I chose PepperScale.com which has the most extensive list that I've found.
* https://www.pepperscale.com/hot-pepper-list/

#### As I failed multiple time to solve this problem myself I reverted to asking the StackOverflow community: https://stackoverflow.com/questions/60327851/trouble-scraping-a-responsive-bootstrap-table/60328204#60328204

The issue comes directly from scraping a bootstrap table that doesn't naturally put it's options (pages, filters, etc) in the browser header. The answer give more detail on his methodology and how he set up this POST request that was neccessary to solve.

##### With this solution you may have to use the Chrome Network Logger to update "wdtNonce" with each browser session

In [3]:
url = "https://www.pepperscale.com/wp-admin/admin-ajax.php?action=get_wdtable&table_id=5"

data = {
    "draw": "2",
    "columns[0][data]": "0",
    "columns[0][name]": "wdt_ID",
    "columns[0][searchable]": "true",
    "columns[0][orderable]": "true",
    "columns[0][search][value]": "",
    "columns[0][search][regex]": "false",
    "columns[1][data]": "1",
    "columns[1][name]": "heat",
    "columns[1][searchable]": "true",
    "columns[1][orderable]": "true",
    "columns[1][search][value]": "",
    "columns[1][search][regex]": "false",
    "columns[2][data]": "2",
    "columns[2][name]": "image",
    "columns[2][searchable]": "true",
    "columns[2][orderable]": "false",
    "columns[2][search][value]": "",
    "columns[2][search][regex]": "false",
    "columns[3][data]": "3",
    "columns[3][name]": "hotpepper",
    "columns[3][searchable]": "true",
    "columns[3][orderable]": "true",
    "columns[3][search][value]": "",
    "columns[3][search][regex]": "false",
    "columns[4][data]": "4",
    "columns[4][name]": "minshu",
    "columns[4][searchable]": "true",
    "columns[4][orderable]": "true",
    "columns[4][search][value]": "",
    "columns[4][search][regex]": "false",
    "columns[5][data]": "5",
    "columns[5][name]": "maxshu",
    "columns[5][searchable]": "true",
    "columns[5][orderable]": "false",
    "columns[5][search][value]": "",
    "columns[5][search][regex]": "false",
    "columns[6][data]": "6",
    "columns[6][name]": "formula_1",
    "columns[6][searchable]": "false",
    "columns[6][orderable]": "false",
    "columns[6][search][value]": "",
    "columns[6][search][regex]": "false",
    "columns[7][data]": "7",
    "columns[7][name]": "formula_2",
    "columns[7][searchable]": "false",
    "columns[7][orderable]": "false",
    "columns[7][search][value]": "",
    "columns[7][search][regex]": "false",
    "columns[8][data]": "8",
    "columns[8][name]": "jalrp",
    "columns[8][searchable]": "true",
    "columns[8][orderable]": "false",
    "columns[8][search][value]": "",
    "columns[8][search][regex]": "false",
    "columns[9][data]": "9",
    "columns[9][name]": "type",
    "columns[9][searchable]": "true",
    "columns[9][orderable]": "true",
    "columns[9][search][value]": "",
    "columns[9][search][regex]": "false",
    "columns[10][data]": "10",
    "columns[10][name]": "origin",
    "columns[10][searchable]": "true",
    "columns[10][orderable]": "false",
    "columns[10][search][value]": "",
    "columns[10][search][regex]": "false",
    "columns[11][data]": "11",
    "columns[11][name]": "use",
    "columns[11][searchable]": "true",
    "columns[11][orderable]": "false",
    "columns[11][search][value]": "",
    "columns[11][search][regex]": "false",
    "columns[12][data]": "12",
    "columns[12][name]": "flavor",
    "columns[12][searchable]": "true",
    "columns[12][orderable]": "false",
    "columns[12][search][value]": "",
    "columns[12][search][regex]": "false",
    "order[0][column]": "5",
    "order[0][dir]": "asc",
    "start": "0",
    "length": "-1",
    "search[value]": "",
    "search[regex]": "false",
    "wdtNonce": "36e9e6949c" #if response is blank update this value (see above)
}

In [4]:
response = requests.post(url, data=data, verify=False)
response.raise_for_status()

# response.text

C:\Users\u104404\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [5]:
peppers = response.json()
peppers = pd.DataFrame(peppers['data'])

In [ ]:
peppers.columns = ("wdt_ID", 'Heat', 'Image', 'Pepper', 'Min SHU', 'Max SHU', 'Median SHU', 'JalRP',
       'JalRP Times Hotter/Milder', 'Type', 'Origin', 'Use', 'Flavor')
peppers

## Prepping the Data

Now that we have the data in it's basic format, we will look at preparing the data for our use. Here are some things we need to accomplish at the step.
1. Formatting the linked content in both the Image columns to just keep the link. 
2. Formatting the Pepper Column into two columns.
    * The first column will have the actually pepper name. 
    * The other with the link to pepper's reference page.
3. Titlecase the 'Type' column to standardize table typecase.
4. Convert numeric columns to correct data types. (Remove thousands separators)
5. (Optional) Decide how to best format the "Flavor" category to make it easily searchable/filterable in our visualization.

### Ad Hoc Changes
* United States and USA

In [ ]:
sorted(peppers['Origin'])

### 1.) Getting the Image Link

In [26]:
# Retrieving the image link for one row
peppers['Image'][3][9:].split('?')[0]

'https://www.pepperscale.com/wp-content/uploads/2015/10/melrose-pepper.jpg'

In [ ]:
peppers['Image'] = peppers["Image"].apply(lambda x: x[9:].split('?')[0])
peppers['Image'][0] = peppers['Image'][0].split("'")[0] # Bell Pepper Exception
peppers

### 2.) Splitting and Formatting the Pepper Name and Reference Page

In [28]:
peppers['Pepper'][61]

"<a data-content='Bishop's Crown Pepper' href='https://www.pepperscale.com/bishops-crown-pepper' target='_blank'>Bishop's Crown Pepper</a>"

In [29]:
peppers['Pepper'][61].split("=")

['<a data-content',
 "'Bishop's Crown Pepper' href",
 "'https://www.pepperscale.com/bishops-crown-pepper' target",
 "'_blank'>Bishop's Crown Pepper</a>"]

In [30]:
# Getting the name
psn = peppers['Pepper'][61].split("=")[1][1:-6]

# Getting the URL
psu = peppers['Pepper'][61].split("=")[2].split("'")[1]

In [31]:
# Splitting the Pepper column for one row
pepper_split = peppers['Pepper'][1].split("=")

pepper_split # Need items 1, and 3

['<a data-content',
 "'Gypsy Pepper' href",
 "'https://www.pepperscale.com/gypsy-pepper' target",
 "'_blank'>Gypsy Pepper</a>"]

In [ ]:
# Capture the link before capturing the pepper name so the link isn't overwritten
peppers['Link'] = peppers["Pepper"].apply(lambda x: x.split("=")[2].split("'")[1])
peppers['Pepper'] = peppers["Pepper"].apply(lambda x: x.split("=")[1][1:-6])
peppers

### 3.) Titlecase the Type Row and Set Index

In [ ]:
peppers['Type'] = peppers['Type'].apply(lambda x: x.title())
peppers.set_index('wdt_ID', inplace=True)
peppers

### 4.) Applying the Correct Data Types to Each Column
#### Resources:
* https://stackoverflow.com/questions/1779288/how-to-convert-a-string-to-a-number-if-it-has-commas-in-it-as-thousands-separato

In [34]:
import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 
locale.atoi('1,000,000')

1000000

In [35]:
peppers['Min SHU'] = peppers['Min SHU'].apply(lambda x: locale.atoi(x))
peppers['Max SHU'] = peppers['Max SHU'].apply(lambda x: locale.atoi(x))
peppers['Median SHU'] = peppers['Median SHU'].apply(lambda x: locale.atoi(x))

peppers.dtypes

Heat                         object
Image                        object
Pepper                       object
Min SHU                       int64
Max SHU                       int64
Median SHU                    int64
JalRP                        object
JalRP Times Hotter/Milder    object
Type                         object
Origin                       object
Use                          object
Flavor                       object
Link                         object
dtype: object

## Exporting the Data

For ease of use in the future we will export this table and reload it going forward. This is mostly to save time because of the verification that is needed when scraping the data that requires to retrieve a specific key. Since the list is still being updated, we will later look at a process to retrieve an updated table every week or month.

In [64]:
peppers.to_csv('outputs\pepper_list.csv')
peppers.to_excel('outputs\pepper_list.xlsx', sheet_name="Pepper List")

### We have completed the data preparation process and can now proceed to Data Exploration and Visualization! Open the next notebook to continue.